## Estimator 이해 및 fit(), predict() 메서드
사이킷런은 ML 모델 학습을 위해서 fit()을 학습된 모델의 예측을 위해 predict() 메서드를 제공
분류 알고리즘을 구현한 Classifier와 회귀 알고리즘을 구현한 Regressor를 제공
Classifier와 Regressor를 합쳐서 Estimator라고 하며 Estimator 클래스는 fit()과 predict()를 내부에서 구현
## Evaluation 함수 및 하이퍼 파라미터 튜닝
cross_val_score()와 같은 evaluation 함수, GridSearchCV와 같은 하이터파라미터 튜닝을 지원하는 클래스의 경우 Estimator를 인자로 받는다.
인자로 받은 Estimator에 대해서 cross_val_score(), GridSearchCV.fit() 함수 내에서 Estimator의 fit()과 predict()를 호출해서 평가를 하거나 하이퍼 파라미터 튜닝을 수행한다.
## 비지도학습과 피처 추출에서 fit()
사이킷런에서 비지도학습인 차원 축소, 클러스터링, 피처 추출등을 구현한 클래스 역시 대부분 fit()과 transform()을 적용한다.
비지도학습과 피처 추출에서 fit()은 지도학습의 fit()과 같이 학습을 의미하는 것이 아니라 입력 데이터의 형태에 맞춰 데이터을 변환하기 위한 사전 구조를 맞추는 작업이다.
fit()으로 변환을 위한 사전 구조를 맞추면 이후 입력 데이터의 차원 변환, 클러스터링, 피처 추출 등의 실제 작업은 transform()으로 수행한다.
fit()과 transform()을 하나로 결합한 fit_transform()도 제공한다.

In [53]:
import pandas as pd
import numpy as np

In [54]:
df = pd.read_csv('./dataset/tdf4.csv')
df.head()

,Unnamed: 0,Survived,Pclass,Sex,Fare,Age_0,Age_1,Age_2,Age_3,Age_4,Embark_0,Embark_1,Embark_2,Family_0,Family_1,Family_2
0,0,0,3,1,1,1,0,0,0,0,0,0,1,0,0,1
1,1,1,1,0,2,1,0,0,0,0,1,0,0,0,0,1
2,2,1,3,0,1,1,0,0,0,0,0,0,1,1,0,0
3,3,1,1,0,2,1,0,0,0,0,0,0,1,0,0,1
4,4,0,3,1,1,1,0,0,0,0,0,0,1,1,0,0


In [58]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

# 독립변수, 종속변수 분리
y = df['Survived']
X = df.drop('Survived',axis=1)
# 학습용 테이터와 평가용 데이터를 8:2로 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10) 

In [59]:
# DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

df_clf = DecisionTreeClassifier()
df_clf.fit(X_train, y_train)
df_clf_pred = df_clf.predict(X_test)

rf_accuracy = accuracy_score(y_test, df_clf_pred)
print('rf 예측 정확도 :', rf_accuracy)

rf 예측 정확도 : 0.7921348314606742


In [57]:
from sklearn.model_selection import KFold
def exec_kfold(clf, folds=5):
# 폴드 세트 5개인 kfold 객체를 생성, 폴드 수만큼 예측 결과 저장을 위한 리스트 생성
    kfold = KFold(n_splits=folds)
    scores = []
    
# KFold 교차검증 수행
    for iter_count, (train_index, test_index) in enumerate(kfold.split(df)):
        X_train, X_test = X.values[train_index], X.values[test_index]
        y_train, y_test = y.values[train_index], y.values[test_index]
        
# classifier 학습, 예측, 정확도 계산
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)
        accuracy = accuracy_score(y_test, pred)
        scores.append(accuracy)
        print("교차 검증 {0} 정확도: {1: .4f}".format(iter_count, accuracy))
        
# 5eo fold에서의 평균 정확도 계산
    mean_score = np.mean(scores)
    print("평균 정확도:{0: 4f}".format(mean_score))
    
# exec_fold 호출
exec_kfold(df_clf, folds=5)

교차 검증 0 정확도:  0.8202
교차 검증 1 정확도:  0.5955
교차 검증 2 정확도:  0.7640
교차 검증 3 정확도:  0.7472
교차 검증 4 정확도:  0.3729
평균 정확도: 0.659970


In [60]:
# 과적합이 일어나는 것을 방지하기 위해 깊이를 조정해준다
from sklearn.model_selection import GridSearchCV
parameters = {'max_depth': [2,3,5,10], 'min_samples_split':[2,3,5], 
             'min_samples_leaf':[1,5,8]}

# cv = cross validation의 약어
grid_dclf = GridSearchCV(df_clf, param_grid=parameters, scoring='accuracy', cv=5)
grid_dclf.fit(X_train, y_train)
print('GCV 최적 하이퍼 파라미터', grid_dclf.best_params_)
print('GCV 최고 정확도:{0: .4f}'.format(grid_dclf.best_score_))
best_dclf = grid_dclf.best_estimator_

# GridSearchCV의 최적 하이퍼 파라미터로 학습된 Estimator로 예측 및 평가 수행
dpred = best_dclf.predict(X_test)
accuracy = accuracy_score(y_test, dpred)
print('DT 정확도 {0: 4f}'.format(accuracy))

GCV 최적 하이퍼 파라미터 {'max_depth': 3, 'min_samples_leaf': 1, 'min_samples_split': 2}
GCV 최고 정확도: 0.8143
DT 정확도  0.831461
